## Tratamiento de datos

In [2]:
import pandas as pd
import numpy as np

### Parquet

In [3]:
df = pd.read_parquet('datasets/vgsales.parquet')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
df.set_index('Publisher', inplace=True)

df.head()

,Rank,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Publisher,,,,,,,,,,
Nintendo,1,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74
Nintendo,2,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24
Nintendo,3,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82
Nintendo,4,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00
Nintendo,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37


In [5]:
df.drop('Rank', axis=1, inplace=True)
df.head()

,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Publisher,,,,,,,,,
Nintendo,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74
Nintendo,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24
Nintendo,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82
Nintendo,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00
Nintendo,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37


In [7]:
# check the nan values
df.isna().sum()

Name              0
Platform          0
Year            271
Genre             0
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [10]:
df.dropna(inplace=True)
df.head()

,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Publisher,,,,,,,,,
Nintendo,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74
Nintendo,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24
Nintendo,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82
Nintendo,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00
Nintendo,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16327 entries, Nintendo to Wanadoo
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          16327 non-null  object 
 1   Platform      16327 non-null  object 
 2   Year          16327 non-null  float64
 3   Genre         16327 non-null  object 
 4   NA_Sales      16327 non-null  float64
 5   EU_Sales      16327 non-null  float64
 6   JP_Sales      16327 non-null  float64
 7   Other_Sales   16327 non-null  float64
 8   Global_Sales  16327 non-null  float64
dtypes: float64(6), object(3)
memory usage: 1.2+ MB


### CSV

In [109]:
indie_games_csv = pd.read_csv('../datasets/indie-games-developers.csv')
indie_games_csv.head()

,Developer,City,Autonomous area,Country,Notable games,Notes
0,11 bit studios,Warsaw,NaN,Poland,AnomalyThis War of MineFrostpunk,NaN
1,ACE Team,Santiago,Santiago,Chile,Zeno ClashRock of Ages,NaN
2,Akupara Games,Los Angeles,California,United States,Whispering Willows The Metronomicon: Slay the ...,NaN
3,Alec Holowka,Winnipeg,Manitoba,Canada,Crayon Physics DeluxeI'm O.K – A Murder Simula...,NaN
4,Alientrap,Saskatoon,Saskatchewan,Canada,CapsizedApotheon,NaN


In [110]:
# comprobar si hay valores nulos 
indie_games_csv.isnull().sum()

Developer            0
City                15
Autonomous area     93
Country              0
Notable games        8
Notes              189
dtype: int64

In [111]:
indie_games_csv = indie_games_csv.drop(columns=['Notes', 'Notable games', 'Autonomous area'])
indie_games_csv.dropna(inplace=True)
indie_games_csv.head()

,Developer,City,Country
0,11 bit studios,Warsaw,Poland
1,ACE Team,Santiago,Chile
2,Akupara Games,Los Angeles,United States
3,Alec Holowka,Winnipeg,Canada
4,Alientrap,Saskatoon,Canada


In [112]:
video_games_csv = pd.read_csv('../datasets/video-games-developers.csv')
video_games_csv = video_games_csv.drop(columns=['Notes', 'Notable games, series or franchises', 'Administrative division', 'Est.'])
video_games_csv.head()

,Developer,City,Country
0,0verflow,Tokyo,Japan
1,11 bit studios,Warsaw,Poland
2,1C Company,Moscow,Russia
3,1-Up Studio,Tokyo,Japan
4,2K Czech,Brno,Czech Republic


### Unión de los csv

In [113]:
# unir los dos dataframes

indie_games_csv = pd.concat([indie_games_csv, video_games_csv], ignore_index=True)
# ordenar por nombre
indie_games_csv.sort_values(by='Developer', inplace=True)
indie_games_csv.head(10)

,Developer,City,Country
185,0verflow,Tokyo,Japan
188,1-Up Studio,Tokyo,Japan
186,11 bit studios,Warsaw,Poland
0,11 bit studios,Warsaw,Poland
187,1C Company,Moscow,Russia
189,2K Czech,Brno,Czech Republic
190,2K Games,Novato,United States
191,2K Sports,Novato,United States
192,343 Industries,Redmond,United States
193,38 Studios,Providence,United States


Filtramos por Developer para eliminar datos redundantes

In [114]:
indie_games_csv = indie_games_csv.drop_duplicates(subset='Developer', keep='first')
indie_games_csv.head(10)

,Developer,City,Country
185,0verflow,Tokyo,Japan
188,1-Up Studio,Tokyo,Japan
186,11 bit studios,Warsaw,Poland
187,1C Company,Moscow,Russia
189,2K Czech,Brno,Czech Republic
190,2K Games,Novato,United States
191,2K Sports,Novato,United States
192,343 Industries,Redmond,United States
193,38 Studios,Providence,United States
194,3D Realms,Garland,United States
